In [1]:
import os
import re
import csv
import pylrc
import shutil
import os.path
import openpyxl
import pandas as pd
from datetime import datetime, date, timedelta

In [2]:
# 檢查資料夾路徑是否存在
url='D:/Anaconda3/Scripts/7 實驗進度/WesternSongs/LrcData_Edit'
print(os.path.exists(url))

True


In [3]:
# 複製指定的lrc檔案到另一個資料夾(LrcModify/lrc)
# https://blog.csdn.net/cheneykl/article/details/79592407
def copyFile(dir1,dir2):
    df_midi = pd.DataFrame()
    list_copyfile_name = []
    for root1, dirs1, file1 in os.walk(dir1):
        for a in file1:
            for root2, dirs2, file2 in os.walk(dir2):
                for b in file2:
                    if os.path.join(a).split('.')[0].replace(' midi download','') == os.path.join(b).split('.')[0]:
                        copyfile = os.path.join(dir2,b)
                        copyfile_name = os.path.basename(copyfile)
                        list_copyfile_name.append(copyfile_name)
                        print(copyfile_name)
                        df_midi = pd.DataFrame(list_copyfile_name)
                        df_midi.to_csv('csv/df_midi.csv',header=0)
                        shutil.copy(os.path.join(dir2,b), r'D:/Anaconda3/Scripts/7 實驗進度/LrcModify/lrc') # lrc複製的目的地
    return df_midi

In [5]:
dir1 = "D:/Anaconda3/Scripts/7 實驗進度/LrcModify/mid" # 目前已下載的mid檔案資料夾
dir2 = "D:/Anaconda3/Scripts/7 實驗進度/WesternSongs/LrcData_Edit" # 挑選lrc的資料夾
copyFile(dir1,dir2)

ABBA_Andante Andante.lrc
ABBA_Angel Eyes.lrc
ABBA_Chiquitita.lrc
ABBA_Dancing Queen.lrc
ABBA_Does Your Mother Know.lrc
ABBA_Fernando.lrc
ABBA_Gimme! Gimme! Gimme!.lrc
ABBA_Happy New Year.lrc
ABBA_Hasta Manana.lrc
ABBA_He is Your Brother.lrc
ABBA_Honey honey.lrc
ABBA_I Do I Do I Do I Do I Do.lrc
ABBA_I Have A Dream.lrc
ABBA_I've Been Waiting For You.lrc
ABBA_Knowing Me Knowing You.lrc
ABBA_Lay All Your Love On Me.lrc
ABBA_Mamma Mia.lrc
ABBA_Money Money Money.lrc
ABBA_My Love My Life.lrc
ABBA_One Of Us.lrc
ABBA_Our Last Summer.lrc
ABBA_People Need Love.lrc
ABBA_Ring Ring.lrc
ABBA_S O S.lrc
ABBA_Slipping Through My Fingers.lrc
ABBA_So Long.lrc
ABBA_Summer Night City.lrc
ABBA_Super Trouper.lrc
ABBA_Take A Chance On Me.lrc
ABBA_Thank You For The Music.lrc
ABBA_That's Me.lrc
ABBA_The Winner Takes It All.lrc
ABBA_Voulez-Vous.lrc
ABBA_Waterloo.lrc
ABBA_When I Kissed The Teacher.lrc
Adam Levine_Go Now.lrc
Adam Levine_Lost Stars.lrc
Adam Levine_No One Else Like You.lrc
Adam Levine_Payphone.lrc
A

,0
0,ABBA_Andante Andante.lrc
1,ABBA_Angel Eyes.lrc
2,ABBA_Chiquitita.lrc
3,ABBA_Dancing Queen.lrc
4,ABBA_Does Your Mother Know.lrc
...,...
76,Alicia Keys_You Don't Know My Name.lrc
77,Andy Williams_ Can't Take My Eyes Off You.lrc
78,Andy Williams_ Love Story.lrc
79,Andy Williams_A Time For Us.lrc


In [4]:
# 匯入歌詞動態時間檔案(.lrc)
def import_lrc(path):
    song = path.split('/')
    filename = song[1]
    print('Filename:',filename) # 印出目前讀取的檔案名稱
    
    # 文字檔逐行讀取
    # https://www.itread01.com/content/1546413512.html
    # https://codertw.com/%E7%A8%8B%E5%BC%8F%E8%AA%9E%E8%A8%80/356561/
    lrc_file = open(path, encoding="utf-8")
    lines = lrc_file.readlines()
    list_lrclines = [] # 文字檔一行一行放入list
    for line in lines:
        #print(line,end="") # 去掉換行符號(print自帶換行)
        list_lrclines.append(line.replace('\xa0',' ').replace('\n','')) # 去掉\xa0、\n換行等符號
    return filename,list_lrclines

In [44]:
path = "lrc/ABBA_Dancing Queen.lrc"
filename,list_lrclines = import_lrc(path)
list_lrclines

Filename: ABBA_Dancing Queen.lrc


['[by:gymnopediee]',
 '[00:20.04]You can dance, you can jive',
 '[00:25.14]Having the time of your life',
 '[00:29.88]See that girl, watch that scene',
 '[00:34.04]Digging the Dancing Queen...',
 '[00:37.18]',
 '[00:44.31]Friday night and the lights are low',
 '[00:49.89]Looking out for the place to go',
 '[00:54.76]Where they play the right music',
 '[00:56.94]Getting in the swing',
 '[00:58.78]You come in to look for a king',
 '[01:03.85]Anybody could be that guy',
 "[01:08.78]Night is young and the music's high",
 '[01:13.24]With a bit of rock music, everything is fine',
 "[01:17.90]You're in the mood for a dance",
 '[01:21.96]And when you get the chance...',
 '[01:25.65]',
 '[01:26.48]You are the Dancing Queen',
 '[01:29.94]Young and sweet, only seventeen',
 '[01:37.42]Dancing Queen, feel the beat from the tambourine',
 '[01:46.65]You can dance, you can jive',
 '[01:51.84]Having the time of your life',
 '[01:56.00]See that girl, watch that scene',
 '[02:00.84]Digging the Dancing Qu

In [34]:
# 讀取原始lrc的內容
def read_content(list_lrclines):
    time_list = []
    lyrics_list = []

    for l in range(len(list_lrclines)):
        # 利用字串的第九個字元判斷目前這時間點的格式，若為「[」代表時間毫秒單位為2位數，反之為3位數
        string = list_lrclines[l]
        
        # 毫秒單位2位數
        if string[9] == ']':
            time = list_lrclines[l][1:9]
            lyrics = list_lrclines[l][10:]
            time_list.append(time)
            lyrics_list.append(lyrics)
        # 毫秒單位3位數
        else:
            time = list_lrclines[l][1:10]
            lyrics = list_lrclines[l][11:]
            time_list.append(time)
            lyrics_list.append(lyrics)
    
    data = {'Time':time_list,'Content':lyrics_list}
    df_lrccontent = pd.DataFrame(data)
    
    # lrc檔案第一行布包括動態歌詞時間點，檢查字串中是否包含指定的子字串{[by:、[ti:、[ar:、[ti:、[al:、(repeat)}
    # 篩選dataframe中含有特定字元的欄位，'|'進行多個條件篩選
    # 「~」代表dataframe反向查詢
    # 替除非時間戳記的列後，篩選產生新的dataframe(df_filter_data)
    # unterminated character set at position錯誤：字串前加入「\」符號
    # https://zhuanlan.zhihu.com/p/29720881
    # https://www.tfzx.net/article/4773721.html
    string = 'by:|ti:|ar:|ti:|al:|repeat'
    df_filter_data = df_lrccontent.loc[~df_lrccontent['Time'].str.contains(string)].reset_index(drop=True)
    
    # 篩選後的df_filter_data，把每一個欄位('Time'和'Content')轉成新的list(time_list_new和lyrics_list_new)
    # https://datatofish.com/convert-pandas-dataframe-to-list/
    time_list_new = df_filter_data['Time'].values.tolist()
    lyrics_list_new = df_filter_data['Content'].values.tolist()
    
    return time_list_new,lyrics_list_new,df_filter_data

In [45]:
time_list_new,lyrics_list_new,df_filter_data = read_content(list_lrclines)
df_filter_data

,Time,Content
0,00:20.04,"You can dance, you can jive"
1,00:25.14,Having the time of your life
2,00:29.88,"See that girl, watch that scene"
3,00:34.04,Digging the Dancing Queen...
4,00:37.18,
5,00:44.31,Friday night and the lights are low
6,00:49.89,Looking out for the place to go
7,00:54.76,Where they play the right music
8,00:56.94,Getting in the swing
9,00:58.78,You come in to look for a king


In [38]:
# 調整lrc動態歌詞的時間點，input時間列表(time_list_new)和第一個音的時間點(space)
def modify_lrc(filename,time_list_new,lyrics_list_new):
    lrc_df = pd.DataFrame()
    t1_list = [] # 陣列儲存lrc第一行的時間點
    t2_list = [] # 陣列儲存lrc第二行的時間點
    t3_list = [] # 陣列儲存mid第一個音的時間點
    result_list = [] # 陣列儲存新lrc的每一個時間點
      
    for j in range(0,len(time_list_new)):
        
        # lrc第一行的時間點
        #print('print(time_list_new[0]) = ', time_list_new[0])
        t1_list.append(time_list_new[0])
        timestep1 = time_list_new[0].split(':')
        t1_minutes = int(timestep1[0])
        t1_seconds = int(timestep1[1][:2])
        #t1_milliseconds = int(timestep1[1][3:])*10
        x = timestep1[1][3:].replace(']','').replace(' ','')
        t1_milliseconds_check = len(x) # 確認毫秒單位的長度
        #print(t1_milliseconds_check)
        if t1_milliseconds_check == 1: # 長度為1
            t1_milliseconds = str(int(x)).zfill(3)
            t1_milliseconds_int = int(x)
        if t1_milliseconds_check == 2: # 長度為2
            x1 = timestep1[1][3:5]
            t1_milliseconds = str(int(x1)*10).zfill(3)
            t1_milliseconds_int = int(x1)
        if t1_milliseconds_check == 3: # 長度為3
            x1 = timestep1[1][3:]
            t1_milliseconds = str(int(x1)).zfill(3) # 字串型態
            t1_milliseconds_int = int(x1) # 轉成數字型態
        #print(t1_minutes, t1_seconds, t1_milliseconds)

        # lrc第二行的時間點
        #print('print(time_list_new[j]) = ',time_list_new[j])
        t2_list.append(time_list_new[j])
        timestep2 = time_list_new[j].split(':')
        t2_minutes = int(timestep2[0])
        t2_seconds = int(timestep2[1][:2])
        #t2_milliseconds = int(timestep2[1][3:])*10
        x = timestep2[1][3:].replace(']','').replace(' ','')
        t2_milliseconds_check = len(x)
        #print(t2_milliseconds_check)
        if t2_milliseconds_check == 1:
            t2_milliseconds = str(int(x)).zfill(3)
            t2_milliseconds_int = int(x)
        if t2_milliseconds_check == 2:
            x1 = timestep2[1][3:5]
            t2_milliseconds = str(int(x1)*10).zfill(3)
            t2_milliseconds_int = int(x1)
        if t2_milliseconds_check == 3:
            x1 = timestep2[1][3:]
            t2_milliseconds = str(int(x1)).zfill(3)
            t2_milliseconds_int = int(x1)
        #print(t2_minutes, t2_seconds, t2_milliseconds)
        
        # mid第一個音的時間
        space = '00:01.660'
        t3_list.append(space)
        timestep3 = space.split(':')
        t3_minutes = int(timestep3[0])
        t3_seconds = int(timestep3[1][:2])
        #t3_milliseconds = int(timestep3[1][3:])*10
        x = timestep3[1][3:].replace(']','').replace(' ','')
        t3_milliseconds_check = len(x)
        #print(t3_milliseconds_check)
        if t3_milliseconds_check == 1:
            t3_milliseconds = str(int(x)).zfill(3)
            t3_milliseconds_int = int(x)
        if t3_milliseconds_check == 2:
            x1 = timestep3[1][3:5]
            t3_milliseconds = str(int(x1)*10).zfill(3)
            t3_milliseconds_int = int(x1)
        if t3_milliseconds_check == 3:
            x1 = timestep3[1][3:]
            t3_milliseconds = str(int(x1)).zfill(3)
            t3_milliseconds_int = int(x1)
        #print(t3_minutes, t3_seconds, t3_milliseconds)
        
        # 時間計算 = 第二行時間點 - 第一行時間點 +第一個音時間點
        t1 = timedelta(minutes=t1_minutes, seconds=t1_seconds, milliseconds=t1_milliseconds_int)
        t2 = timedelta(minutes=t2_minutes, seconds=t2_seconds, milliseconds=t2_milliseconds_int)
        t3 = timedelta(minutes=t3_minutes, seconds=t3_seconds, milliseconds=t3_milliseconds_int)
        x = t2 - t1 + t3
        
        # 格式化新時間點，並放入list(result_list)
        a = str(x).split(':')
        minute = a[1]
        second = a[2][:2]
        millisecond = a[2][3:5]
        #print(minute, second, str(millisecond).zfill(2))
        result = '['+ minute + ':' + second + '.' + str(millisecond).zfill(2) + '0]'
        #print(result)
        result_list.append(result)
        #print('-'*50)
    
    # 放入dataframe中(lrc_df)
    lrc_df['t1'] = pd.Series(t1_list)
    lrc_df['t2'] = pd.Series(t2_list)
    lrc_df['t3'] = pd.Series(t3_list)
    lrc_df['result'] = pd.Series(result_list)
    lrc_df['lyrics'] = pd.Series(lyrics_list_new)

    # 輸出計算時間差並加上每一行歌詞新的時間點的csv
    lrc_path_01 = 'csv/fix_lrc/lrc_df_' + filename
    lrc_path_01 = lrc_path_01.replace('.lrc','.xlsx')
    print('Save in ' + lrc_path_01)
    lrc_df.to_excel(lrc_path_01, index=False, encoding='utf_8_sig')
    return lrc_df

In [46]:
lrc_df = modify_lrc(filename,time_list_new,lyrics_list_new)
lrc_df

Save in csv/fix_lrc/lrc_df_ABBA_Dancing Queen.xlsx


,t1,t2,t3,result,lyrics
0,00:20.04,00:20.04,00:01.660,[00:01.660],"You can dance, you can jive"
1,00:20.04,00:25.14,00:01.660,[00:06.670],Having the time of your life
2,00:20.04,00:29.88,00:01.660,[00:10.740],"See that girl, watch that scene"
3,00:20.04,00:34.04,00:01.660,[00:15.660],Digging the Dancing Queen...
4,00:20.04,00:37.18,00:01.660,[00:18.670],
5,00:20.04,00:44.31,00:01.660,[00:25.680],Friday night and the lights are low
6,00:20.04,00:49.89,00:01.660,[00:30.740],Looking out for the place to go
7,00:20.04,00:54.76,00:01.660,[00:35.730],Where they play the right music
8,00:20.04,00:56.94,00:01.660,[00:37.750],Getting in the swing
9,00:20.04,00:58.78,00:01.660,[00:39.730],You come in to look for a king


In [75]:
# 輸出新的lrc檔案到lrc_new資料夾
def output_newlrc(filename,lrc_df):
    # 只印特定欄位有歌詞
    lrc_df2 = lrc_df[['result', 'lyrics']]
    lrc_path_02 = 'lrc_new/lrc_' + filename
    print('Save in ' + lrc_path_02)
    
    # 篩選lyrics非空值的列，刪除為空值的列，另存為新dataframe(lrc_df3)
    lrc_df3 = lrc_df2.loc[lrc_df2['lyrics'] != '']
    lrc_df3.reset_index(drop=True)
    
    value_list = [] # 陣列儲存要寫入的
    for i in range(len(lrc_df3)):
        # 有歌詞
        value = lrc_df3.iat[i,0] + lrc_df3.iat[i,1]
        # 沒歌詞
        #value = lrc_df3.iat[i, 0]
        value_list.append(value)
    
    # 逐行寫入lrc檔案
    # https://blog.csdn.net/matrix_google/article/details/76861485
    file_write_obj = open(lrc_path_02,'w',encoding='utf_8_sig')
    for var in value_list:
        file_write_obj.writelines(var)
        file_write_obj.write('\n')
    file_write_obj.close()

    # 取代歌詞欄位的「′」符號
    text = open(lrc_path_02, "r", encoding='utf_8_sig')
    text = ''.join([i for i in text]).replace("′", "'")
    edit = open(lrc_path_02,"w",encoding='utf_8_sig')
    edit.writelines(text)
    edit.close()
    
    return value_list,lrc_df3

In [76]:
value_list,lrc_df3 = output_newlrc(filename,lrc_df)
lrc_df3

Save in lrc_new/lrc_ABBA_Dancing Queen.lrc


,result,lyrics
0,[00:01.660],"You can dance, you can jive"
1,[00:06.670],Having the time of your life
2,[00:10.740],"See that girl, watch that scene"
3,[00:15.660],Digging the Dancing Queen...
5,[00:25.680],Friday night and the lights are low
6,[00:30.740],Looking out for the place to go
7,[00:35.730],Where they play the right music
8,[00:37.750],Getting in the swing
9,[00:39.730],You come in to look for a king
10,[00:44.740],Anybody could be that guy


In [77]:
# 主程式執行
path = "lrc/ABBA_Does Your Mother Know.lrc"
filename,list_lrclines = import_lrc(path)
time_list_new,lyrics_list_new,df_filter_data = read_content(list_lrclines)
lrc_df = modify_lrc(filename,time_list_new,lyrics_list_new)
value_list,lrc_df3 = output_newlrc(filename,lrc_df)

Filename: ABBA_Does Your Mother Know.lrc
Save in csv/fix_lrc/lrc_df_ABBA_Does Your Mother Know.xlsx
Save in lrc_new/lrc_ABBA_Does Your Mother Know.lrc


In [143]:
time_list = ['00:06.550','01:12.04','01:12.60','00:21.90]','01:57.0]','02:12.050','03:25.102']
timestep2 = time_list[1].split(':')

t2_minutes = int(timestep2[0])
t2_seconds = int(timestep2[1][:2])

x = timestep2[1][3:].replace(']','').replace(' ','')
t2_milliseconds_check = len(x)
print(t2_milliseconds_check)

if t2_milliseconds_check == 1:
    t2_milliseconds = str(int(x)).zfill(3)
    t2_milliseconds_int = int(x)
if t2_milliseconds_check == 2:
    x1 = timestep2[1][3:5]
    t2_milliseconds = str(int(x1)*10).zfill(3)
    t2_milliseconds_int = int(x1)
if t2_milliseconds_check == 3:
    x1 = timestep2[1][3:]
    t2_milliseconds = str(int(x1)).zfill(3)
    t2_milliseconds_int = int(x1)

print(t2_minutes, t2_seconds, t2_milliseconds)

2
1 12 040
